## 一般的mnist和一般的cnn

In [18]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=0)


model = Net()
criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

for epoch in range(1, 21):
    train(epoch)
    test(epoch)
print('done')


1:	Test set: Average loss: 0.0032, Accuracy: 9376/10000 (93%)
2:	Test set: Average loss: 0.0022, Accuracy: 9585/10000 (95%)
3:	Test set: Average loss: 0.0016, Accuracy: 9693/10000 (96%)
4:	Test set: Average loss: 0.0014, Accuracy: 9731/10000 (97%)
5:	Test set: Average loss: 0.0012, Accuracy: 9782/10000 (97%)
6:	Test set: Average loss: 0.0011, Accuracy: 9796/10000 (97%)
7:	Test set: Average loss: 0.0010, Accuracy: 9802/10000 (98%)
8:	Test set: Average loss: 0.0010, Accuracy: 9804/10000 (98%)
9:	Test set: Average loss: 0.0009, Accuracy: 9820/10000 (98%)
10:	Test set: Average loss: 0.0009, Accuracy: 9822/10000 (98%)
11:	Test set: Average loss: 0.0009, Accuracy: 9824/10000 (98%)
12:	Test set: Average loss: 0.0009, Accuracy: 9833/10000 (98%)
13:	Test set: Average loss: 0.0008, Accuracy: 9827/10000 (98%)
14:	Test set: Average loss: 0.0008, Accuracy: 9829/10000 (98%)
15:	Test set: Average loss: 0.0008, Accuracy: 9839/10000 (98%)
16:	Test set: Average loss: 0.0008, Accuracy: 9840/10000 (98%)
1

## GPU Enabled

In [19]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Check if cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=0)


model = Net()
model = model.to(device)
criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.001)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        # sum up batch loss
        #test_loss += F.nll_loss(output, target, reduction='sum').item()
        test_loss += criterion(output, target).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 21):
    train(epoch)
    test(epoch)
print('done')


1:	Test set: Average loss: 0.0031, Accuracy: 9403/10000 (94%)
2:	Test set: Average loss: 0.0021, Accuracy: 9601/10000 (96%)
3:	Test set: Average loss: 0.0016, Accuracy: 9666/10000 (96%)
4:	Test set: Average loss: 0.0014, Accuracy: 9735/10000 (97%)
5:	Test set: Average loss: 0.0013, Accuracy: 9730/10000 (97%)
6:	Test set: Average loss: 0.0011, Accuracy: 9771/10000 (97%)
7:	Test set: Average loss: 0.0011, Accuracy: 9778/10000 (97%)
8:	Test set: Average loss: 0.0010, Accuracy: 9789/10000 (97%)
9:	Test set: Average loss: 0.0009, Accuracy: 9817/10000 (98%)
10:	Test set: Average loss: 0.0009, Accuracy: 9822/10000 (98%)
11:	Test set: Average loss: 0.0009, Accuracy: 9813/10000 (98%)
12:	Test set: Average loss: 0.0009, Accuracy: 9829/10000 (98%)
13:	Test set: Average loss: 0.0009, Accuracy: 9816/10000 (98%)
14:	Test set: Average loss: 0.0009, Accuracy: 9817/10000 (98%)
15:	Test set: Average loss: 0.0009, Accuracy: 9828/10000 (98%)
16:	Test set: Average loss: 0.0008, Accuracy: 9841/10000 (98%)
1

## 加了L1

In [14]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Check if cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=0)


model = Net()
model = model.to(device)
criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.001)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        regularization_loss = 0
        for param in model.parameters():
            regularization_loss += torch.mean(abs(param))
        
        loss = criterion(output, target)
        loss += 0.01 * regularization_loss # lamda = 0.01
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        # sum up batch loss
        #test_loss += F.nll_loss(output, target, reduction='sum').item()
        test_loss += criterion(output, target).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 21):
    train(epoch)
    test(epoch)
print('done')


1:	Test set: Average loss: 0.0033, Accuracy: 9351/10000 (93%)
2:	Test set: Average loss: 0.0021, Accuracy: 9610/10000 (96%)
3:	Test set: Average loss: 0.0017, Accuracy: 9662/10000 (96%)
4:	Test set: Average loss: 0.0014, Accuracy: 9719/10000 (97%)
5:	Test set: Average loss: 0.0013, Accuracy: 9736/10000 (97%)
6:	Test set: Average loss: 0.0012, Accuracy: 9739/10000 (97%)
7:	Test set: Average loss: 0.0011, Accuracy: 9761/10000 (97%)
8:	Test set: Average loss: 0.0011, Accuracy: 9775/10000 (97%)
9:	Test set: Average loss: 0.0010, Accuracy: 9796/10000 (97%)
10:	Test set: Average loss: 0.0010, Accuracy: 9778/10000 (97%)
11:	Test set: Average loss: 0.0010, Accuracy: 9804/10000 (98%)
12:	Test set: Average loss: 0.0009, Accuracy: 9814/10000 (98%)
13:	Test set: Average loss: 0.0009, Accuracy: 9822/10000 (98%)
14:	Test set: Average loss: 0.0009, Accuracy: 9816/10000 (98%)
15:	Test set: Average loss: 0.0008, Accuracy: 9833/10000 (98%)
16:	Test set: Average loss: 0.0009, Accuracy: 9824/10000 (98%)
1

## 加了L2

In [27]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Check if cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)
        

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=0)


model = Net()
model = model.to(device)
criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.001)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 21):
    train(epoch)
    test(epoch)
print('done')

1:	Test set: Average loss: 0.0032, Accuracy: 9395/10000 (93%)
2:	Test set: Average loss: 0.0020, Accuracy: 9591/10000 (95%)
3:	Test set: Average loss: 0.0015, Accuracy: 9697/10000 (96%)
4:	Test set: Average loss: 0.0013, Accuracy: 9747/10000 (97%)
5:	Test set: Average loss: 0.0011, Accuracy: 9776/10000 (97%)
6:	Test set: Average loss: 0.0011, Accuracy: 9778/10000 (97%)
7:	Test set: Average loss: 0.0010, Accuracy: 9796/10000 (97%)
8:	Test set: Average loss: 0.0010, Accuracy: 9808/10000 (98%)
9:	Test set: Average loss: 0.0010, Accuracy: 9804/10000 (98%)
10:	Test set: Average loss: 0.0009, Accuracy: 9813/10000 (98%)
11:	Test set: Average loss: 0.0009, Accuracy: 9826/10000 (98%)
12:	Test set: Average loss: 0.0009, Accuracy: 9829/10000 (98%)
13:	Test set: Average loss: 0.0009, Accuracy: 9835/10000 (98%)
14:	Test set: Average loss: 0.0009, Accuracy: 9820/10000 (98%)
15:	Test set: Average loss: 0.0008, Accuracy: 9839/10000 (98%)
16:	Test set: Average loss: 0.0008, Accuracy: 9829/10000 (98%)
1